In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.0'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.c

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.5.0.jar

--2022-10-08 20:59:00--  https://jdbc.postgresql.org/download/postgresql-42.5.0.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1046274 (1022K) [application/java-archive]
Saving to: ‘postgresql-42.5.0.jar’

postgresql-42.5.0.j 100%[===================>]   1022K  5.47MB/s    in 0.2s    

2022-10-08 20:59:00 (5.47 MB/s) - ‘postgresql-42.5.0.jar’ saved [1046274/1046274]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.5.0.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!|2015-08-31 00:00:00|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...| 

In [11]:
df_total_Votes = df.filter("total_votes >= 20")
df_total_Votes.show()

In [17]:
df_votes = df_total_Votes.withColumn("helpfulByTotal", df_total_Votes["helpful_votes"]/df_total_Votes["total_votes"]).filter("helpful_votes/total_votes > 0.50")
df_votes.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|    helpfulByTotal|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|         US|   11025729| ROP6ITXO8K5V2|B00LE9MZMW|     778942419|YJ Moyu 13x13x13 ...|            Toys|          5|           23|         27|   N|                Y|          Five Stars|Great cube, moves...|2015-08-31 00:00:00|0.8518518518518519|
|         US

In [32]:
df_vine_paid = df_votes.filter("vine='Y'")
df_vine_paid.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|    helpfulByTotal|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|         US|   53067113| R9I5FOLKU99RY|B00U1KA12G|     278300201|Qixels S1 Turbo D...|            Toys|          5|           19|         20|   Y|                N|11 Year Old Loves...|My daughter who i...|2015-08-31 00:00:00|              0.95|
|         US

In [33]:
df_vine_not_paid = df_votes.filter("vine='N'")
df_vine_not_paid.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|    helpfulByTotal|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|         US|   11025729| ROP6ITXO8K5V2|B00LE9MZMW|     778942419|YJ Moyu 13x13x13 ...|            Toys|          5|           23|         27|   N|                Y|          Five Stars|Great cube, moves...|2015-08-31 00:00:00|0.8518518518518519|
|         US

In [35]:
#Determine total paid reviews & unpaid reviews

total_paid_reviews = df_vine_paid.count()
total_not_paid_reviews = df_vine_not_paid.count()

total_paid_5star_reviews = df_vine_paid.filter("star_rating=5").count()
total_unpaid_5star_reviews = df_vine_not_paid.filter("star_rating=5").count()

In [37]:
print(f"Paid Total reviews: {total_paid_reviews}")
print(f"Paid 5-star reviews: {total_paid_5star_reviews}")
print(f"Unpaid Total reviews: {total_not_paid_reviews}")
print(f"Unpaid 5-star reviews: {total_unpaid_5star_reviews}")
print(f"% of Paid 5-star Reviews: {(total_paid_5star_reviews/total_paid_reviews)*100}%")
print(f"% of Unpaid 5-star Reviews: {(total_unpaid_5star_reviews/total_not_paid_reviews)*100}%")



Paid Total reviews: 1266
Paid 5-star reviews: 432
Unpaid Total reviews: 61849
Unpaid 5-star reviews: 29950
% of Paid 5-star Reviews: 34.12322274881517%
% of Unpaid 5-star Reviews: 48.42438842988569%
